In [ ]:
import socket

# Server configuration
ip_pool = ["192.168.1.100", "192.168.1.101", "192.168.1.102"]
leases = {}

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1)  # Allow 1 pending connection

print("TCP Server is listening...")

while True:
    client_socket, client_address = server_socket.accept()
    print(f"Connected to {client_address}")
    
    # Step 1: Receive DHCP DISCOVER
    discover_message = client_socket.recv(1024).decode()
    print(f"Received: {discover_message}")
    
    if discover_message == "DISCOVER":
        # Step 2: Send DHCP OFFER
        if ip_pool:
            offered_ip = ip_pool.pop(0)
            offer_message = f"OFFER {offered_ip}"
            print(f"Sending: {offer_message}")
            client_socket.sendall(offer_message.encode())
        
        # Step 3: Receive DHCP REQUEST
        request_message = client_socket.recv(1024).decode()
        print(f"Received: {request_message}")
        
        if request_message.startswith("REQUEST"):
            assigned_ip = request_message.split()[1]
            leases[client_address] = assigned_ip
            
            # Step 4: Send DHCP ACK
            ack_message = f"ACK {assigned_ip}"
            print(f"Sending: {ack_message}")
            client_socket.sendall(ack_message.encode())

    client_socket.close()
